In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D,AlphaDropout,SeparableConv2D, SpatialDropout2D, MaxPooling2D
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
%matplotlib inline

In [ ]:
!pip install git+https://github.com/rcmalli/keras-vggface.git -q

In [ ]:
from keras_vggface.vggface import VGGFace
vgg = VGGFace(model='vgg16',include_top=False, input_shape=(200, 200, 3))

104947712/104944616 [==============================] - 4s 0us/step


In [ ]:
fx_train_sen = vgg.predict(x_train)
fx_val_sen = vgg.predict(x_val)
fx_test_sen = vgg.predict(x_test)

In [ ]:
#Loading Labels
traindf = pd.read_excel('/content/drive/My Drive/Facial Demographics/traindf.xlsx')
valdf = pd.read_excel('/content/drive/My Drive/Paper/Facial Demographics/valdf.xlsx')
testdf = pd.read_excel('/content/drive/My Drive/Paper/Facial Demographics/testdf.xlsx')
traindf.head()

In [ ]:
traindf['gender'].value_counts()

Gender Classification Model

In [ ]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',np.unique(traindf['gender'].values),traindf['gender'].values)
class_weights

array([0.95762626, 1.04629732])

In [ ]:
model = Sequential()

model.add(BatchNormalization())
model.add(SpatialDropout2D(0.5))
model.add(SeparableConv2D(512,(3,3),padding='same'))
model.add(MaxPooling2D((2,2)))

model.add(BatchNormalization())
model.add(SpatialDropout2D(0.5))
model.add(SeparableConv2D(512,(3,3),padding='same'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(BatchNormalization())
model.add(AlphaDropout(0.5))
model.add(Dense(128,activation='relu',kernel_initializer='he_uniform'))

model.add(BatchNormalization())
model.add(Dense(1,activation='sigmoid'))

In [ ]:
sgd = tf.keras.optimizers.SGD(
    learning_rate=1e-4, momentum=0, nesterov=False, name="SGD")

adam = tf.keras.optimizers.Adam(
    learning_rate=1e-3,
    amsgrad=True,)

adamax = tf.keras.optimizers.Adamax(learning_rate=1e-4)

model.compile(optimizer=adam,loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
!pip install keras-adabound -q

In [ ]:
model.fit(fx_train_vgg,traindf['gender'].values,epochs=10, validation_data=(fx_val_vgg,valdf['gender'].values),class_weight=class_weights,batch_size=64)

In [ ]:
model.evaluate(fx_test_vgg,testdf['gender'].values)

In [ ]:
model.predict(fx_test_vgg).shape

DO the same loop for all the 4 networks and ensamble in the next cell

Ensemble

In [ ]:
from keras.models import load_model

G_res = load_model('/content/drive/My Drive/Facial Demographics/resnet.h5')
G_res.compile("Adam","binary_crossentropy",['acc'])

G_sen = load_model('/content/drive/My Drive/Facial Demographics/senet.h5')
G_sen.compile("Adam","binary_crossentropy",['acc'])

G_vgg = load_model('/content/drive/My Drive/Facial Demographics/vgg.h5')
G_vgg.compile("Adam","binary_crossentropy",['acc'])

In [ ]:
fx_test_res_e = np.load('/content/drive/My Drive/Paper/UTKFace/fx_test_res.npy')
fx_test_sen_e = np.load('/content/drive/My Drive/Paper/UTKFace/fx_test_sen.npy')
fx_test_vgg_e = np.load('/content/drive/My Drive/Paper/UTKFace/fx_test_vgg.npy')

In [ ]:
pred_res = G_res.predict(fx_test_res_e)
pred_sen = G_sen.predict(fx_test_sen_e)
pred_vgg = G_vgg.predict(fx_test_vgg_e)

In [ ]:
pred_res.shape, pred_sen.shape, pred_vgg.shape

In [ ]:
y_pred = (pred_res+pred_sen+pred_vgg)/3

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(testdf['gender'].values,y_pred.round())
print(report)

In [ ]:
from sklearn.metrics import roc_auc_score

auc_vgg = roc_auc_score(testdf['gender'].values,pred_vgg.round())
auc_res = roc_auc_score(testdf['gender'].values,pred_res.round())
auc_sen = roc_auc_score(testdf['gender'].values,pred_sen.round())
auc_en = roc_auc_score(testdf['gender'].values,y_pred.round())
print('ROC_AUC_SCORE : \n')
print('ResNet50: ',auc_res)
print('SeNet50: ',auc_sen)
print('VGG16: ',auc_vgg)
print('Ensemble: ',auc_en)

In [ ]:
from sklearn.metrics import accuracy_score 

acc_vgg = accuracy_score(testdf['gender'].values,pred_vgg.round())
acc_res = accuracy_score(testdf['gender'].values,pred_res.round())
acc_sen = accuracy_score(testdf['gender'].values,pred_sen.round())
acc_en = accuracy_score(testdf['gender'].values,y_pred.round())
print('ACCURACY_SCORE : \n')
print('ResNet50: ',acc_res)
print('SeNet50: ',acc_sen)
print('VGG16: ',acc_vgg)
print('Ensemble: ',acc_en)